In [ ]:
import numpy as np
import CHONK_XL as chxl
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import xsimlab as xs
import CHONK_cpp as ch

import zarr
import helplotlib as hpl
import helper
%matplotlib widget
%load_ext xsimlab.ipython

In [ ]:


@xs.process
class CustomParameters:
    label_array = xs.variable(intent = 'out', dims = (('y','x'), ('node')))
    label_list = xs.any_object()
    CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
    nx = xs.foreign(chxl.ChonkBase, "nx")
    ny = xs.foreign(chxl.ChonkBase, "ny")
    dx = xs.foreign(chxl.ChonkBase, "dx")
    dy = xs.foreign(chxl.ChonkBase, "dy")
    
    active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
#     landscape = xs.any_object()

    def initialize(self):

        self.label_list = []
    
        self.label_array = np.zeros((nx*ny), dtype = np.int)

        self.label_list.append(ch.label(0))
        self.label_list[-1].m = 0.45;
        self.label_list[-1].n = 1;
        self.label_list[-1].base_K = 1e-5;
        self.label_list[-1].Ks_modifyer = 1.2;
        self.label_list[-1].Kr_modifyer = 0.8;
        self.label_list[-1].dimless_roughness = 0.5;
        self.label_list[-1].V = 0.5;
        self.label_list[-1].dstar = 1;
        self.label_list[-1].threshold_incision = 0;
        self.label_list[-1].threshold_entrainment = 0;
        self.label_list[-1].kappa_base = 1e-4;
        self.label_list[-1].kappa_r_mod = 0.8;
        self.label_list[-1].kappa_s_mod = 1.2;
        self.label_list[-1].critical_slope = 0.57835;
        self.label_list[-1].sensitivity_tool_effect = 1;

        self.label_list.append(ch.label(1))
        self.label_list[-1].m = 0.45;
        self.label_list[-1].n = 1;
        self.label_list[-1].base_K = 1e-5;
        self.label_list[-1].Ks_modifyer = 1;
        self.label_list[-1].Kr_modifyer = 0.3;
        self.label_list[-1].dimless_roughness = 0.5;
        self.label_list[-1].V = 0.1;
        self.label_list[-1].dstar = 1;
        self.label_list[-1].threshold_incision = 0;
        self.label_list[-1].threshold_entrainment = 0;
        self.label_list[-1].kappa_base = 1e-4;
        self.label_list[-1].kappa_r_mod = 0.8;
        self.label_list[-1].kappa_s_mod = 1.2;
        self.label_list[-1].critical_slope = 0.57835;
        self.label_list[-1].sensitivity_tool_effect = 1;

        self.CHONK.initialise_label_list(self.label_list)
        self.CHONK.update_label_array(self.label_array.ravel())
        
        
@xs.process
class UpliftLandscape(chxl.Uplift):
	uplift_done = xs.variable(intent = "out")
	runner_done = xs.foreign(chxl.Runner, "runner_done")
	uplift = xs.variable(intent = 'out', dims = [('y','x'), ('node')])
	switch_time = xs.variable(intent = 'in')
	CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
	active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
	nx = xs.foreign(chxl.ChonkBase, "nx")
	ny = xs.foreign(chxl.ChonkBase, "ny")
	dx = xs.foreign(chxl.ChonkBase, "dx")
	dy = xs.foreign(chxl.ChonkBase, "dy")
    
	rate = xs.variable()
	idx_N1 = xs.variable()
	idx_N2 = xs.variable()
	idx_N3 = xs.variable()
	idx_N4 = xs.variable()
    

	def initialize(self):
		self.uplift = np.zeros((ny,nx))
		self.uplift[self.idx_N1:self.idx_N2,:] = 1
		self.uplift[self.idx_N3:self.idx_N4,:] = 1
		from scipy.ndimage import gaussian_filter
		self.uplift = gaussian_filter(self.uplift, sigma=2)
		self.uplift /= self.uplift.max()
		self.uplift *= self.rate
		self.uplift[[-1,0],:] = 0
		self.done = False

	@xs.runtime(args=['step_delta','step_end'])
	def run_step(self, dt, timing):
		self.CHONK.add_external_to_surface_elevation_tp1(self.uplift.ravel() * dt)
		self.uplift_done = True
		
# landscape.uplift4StSt

In [ ]:
model = xs.Model({"ChonkBase": chxl.ChonkBase,
                "Runner": chxl.Runner,
                "Topography": chxl.Topography,
#                 "Uplift": UpliftLandscapeStSt,
                "Uplift": UpliftLandscape,
#                 "Uplift": UpliftLandscapeNF1,
#                 "Uplift": chxl.Uplift,
                "Lake": chxl.Lake,
                "Precipitation": chxl.Precipitation,
#                 "DefaultParameters": chxl.DefaultParameters,
                "DefaultParameters": CustomParameters,
                "Flow": chxl.Flow,
                "Fluvial": chxl.Fluvial,
#                 "Hillslope": chxl.Hillslope
            })



In [ ]:
ny,nx = 100,30
dy,dx = 200,200
init_z = np.random.rand(ny,nx)
# np.save("WB_checker",init_z)

In [ ]:
# %create_setup model
import xsimlab as xs
time = np.arange(0,1e6,1000)
# time = [0,1000,2000]
otime = time[::10]

init_z = np.load("./WB_checker.npy")
init_z = np.random.rand(ny,nx)
init_z[[-1,0],:] -=1 
init_z+=1

rate = 0.5e-3
idx_N1 = 10
idx_N2 = 30
idx_N3 = 70
idx_N4 = 80
# init_z[0,:] = 1000
# init_z[0,:] = 0
U = np.zeros((ny,nx)) + 2e-4

Utime = 1e8
ds_in = xs.create_setup(
    model=model,
    clocks={
        'time': time,
        'otime': otime
    },
    master_clock='time',
    input_vars={
        'ChonkBase__dx': dx,
        'ChonkBase__dy': dy,
        'ChonkBase__nx': nx,
        'ChonkBase__ny': ny,
        'ChonkBase__depths_res_sed_proportions': 10,
        'ChonkBase__n_depth_sed_tracking': 50,
        'ChonkBase__boundary_conditions': "periodic_EW",
        'Topography__initial_elevation': init_z,
        'Uplift__rate': rate,
        'Uplift__idx_N1': idx_N1,
        'Uplift__idx_N2': idx_N2,
        'Uplift__idx_N3': idx_N3,
        'Uplift__idx_N4': idx_N4,
        'Uplift__switch_time': Utime,
        'Lake__method': 'explicit', # ('batch', ['implicit', 'explicit']),
        'Lake__evaporation': True,
        'Lake__evaporation_rate': 1.4,#('batch', [1e-2,0.1,1,2,5,10, 100]),
        'Flow__threshold_single_flow': 1e9,
        'Precipitation__precipitation_rate': 1
    },
    output_vars={
        'Topography__topography': 'otime',
        'Topography__sed_height': 'otime',
        'Flow__Qw': 'otime',
        'Flow__water_balance_checker': 'time',
        'Lake__lake_depth': 'otime',
        'Fluvial__Qs': 'otime',
        'ChonkBase__D_s': 'otime',
        'ChonkBase__E_s': 'otime',
        'ChonkBase__E_r': 'otime',
        'Lake__topolake': 'otime',
#         'Hillslope__Qs': 'otime',
    }
)


In [ ]:
# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=1)  
zg = zarr.group("WBchecekr.zarr", overwrite=True)
with model,xs.monitoring.ProgressBar():
    out_ds = ds_in.xsimlab.run(store = zg)
#     out_ds = mod1.xsimlab.run()  
out_ds.x.values[0] = 0
out_ds.y.values[0] = 0
# out_ds["topolake"] = out_ds["Topography__topography"] + out_ds["Lake__lake_depth"]

In [ ]:
fig,ax = plt.subplots()
ax2 = ax.twinx()
ax2.plot(out_ds.time.values, out_ds.Flow__water_balance_checker.values/ (out_ds.Topography__topography.values[-1].ravel().shape[0] * dx * dy) * 100, color = 'r')
ax2.set_ylabel("% watbal")
ax.plot(out_ds.time.values, out_ds.Flow__water_balance_checker.values,color = 'blue' )

In [ ]:
from ipyfastscape import TopoViz3d


app = TopoViz3d(out_ds, canvas_height=600, time_dim="otime", elevation_var = "Lake__topolake" )
# app = TopoViz3d(out_ds, canvas_height=600, time_dim="otime", elevation_var = "Topography__topography" )
# 

app.show()

In [ ]:
np.save("WB_checker.npy", out_ds.Topography__topography.values[-1])
# np.save("sed4testHS_CHONK.npy", out_ds.Topography__sed_height.sel({'otime':1.001e6}, method="nearest").values)
# np.save("sed4testtopo_CHONK.npy", out_ds.Topography__topography.sel({'otime':1.001e6}, method="nearest").values)

In [ ]:
fig,ax = plt.subplots()
# ax.imshow(np.log10(out_ds.Flow__Qw.values[0]))
# cb = ax.imshow(np.load("Stuff.npy").reshape(ny,nx))
cb = ax.imshow(out_ds.Lake__lake_depth.values[36], vmin = 0, vmax = 0.1)
plt.colorbar(cb)

In [ ]:
from importlib import reload  
import plotter as pol
import warnings;warnings.simplefilter('ignore')
reload(pol)

pol.anim_lake_cross_section(
	out_ds, # The input ds 
	fname = "outputgif",
	timedim = "otime", # the time dimension
	batch_dim = None, # if there is a batch dim to pick
	cross_section_dir = 'x', # is the cross section in x or y direction
	xy_cross_section = 10000, # coordinate on the other axis
	color_bedrock = 'gray', # color of the bedrock
	color_sediments = 'orange', # color of the bedrock
	color_water = 'blue', # color of the water
	z_min = None, # minimum z on the cross_section, if left to none -> min of all
	z_max = None, # max z on the cross_section, if left to none -> min of all
	# Map parameters
	cmap_elev = 'gist_earth', # cmap of the cross-section
	alpha_hillshade = 0.5, # transparency of the hillshade
    czmin = 0,
    czmax = 1500,
    figsize = (12,4.5),
    custom_tickszz = np.arange(0,21000,5000, dtype = np.int),
)

In [ ]:
A =np.zeros((1092 * 1002), dtype= np.float32)

In [ ]:
A.shape

In [ ]:
A.nbytes * 5 * 200

In [ ]:
4376736000 / 1e6

In [ ]:
from ipyfastscape import TopoViz3d
import xarray as xr
out_ds = xr.open_zarr("rift_lake_evaporation_2.zarr")
out_ds.x.values[0] = 0
out_ds.y.values[0] = 0
# app = TopoViz3d(out_ds.load(), canvas_height=600, time_dim="otime", elevation_var = "Lake__topolake" )
app = TopoViz3d(out_ds, canvas_height=600, time_dim="otime", elevation_var = "Topography__topography" )
# 

app.show()

In [ ]:
out_ds["Topography__topography"]